In [25]:
import ee
import numpy as np

# Trigger the authentication flow.
# ee.Authenticate()

ee.Initialize(project='ee-arzaaan789')



In [40]:
import pandas as pd
from tqdm import tqdm
# Load and filter data
def load_filter_data(filepath):
    df = pd.read_csv(filepath, delimiter='\t')
    df = df[df["occurrenceStatus"] == "PRESENT"]
    return df


# Load all data at once
files = ['Muscardinus avellanarius.csv']
dfs = [load_filter_data(f) for f in files]
df = pd.concat(dfs, ignore_index=True)
len(df[df['year']>=2024])

/var/folders/0n/sggn66691kd6g5mtyzpq2hv80000gn/T/ipykernel_18706/924961457.py:5: DtypeWarning: Columns (10,16,37,38,39,40,41,46,48) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath, delimiter='\t')


2037

In [3]:
from pyproj import Transformer
from rasterio.windows import Window

land_cover_map = {
    1: "Deciduous woodland",
    2: "Coniferous woodland",
    3: "Arable",
    4: "Improved grassland",
    5: "Neutral grassland",
    6: "Calcareous grassland",
    7: "Acid grassland",
    8: "Fen",
    9: "Heather",
    10: "Heather grassland",
    11: "Bog",
    12: "Inland rock",
    13: "Saltwater",
    14: "Freshwater",
    15: "Supralittoral rock",
    16: "Supralittoral sediment",
    17: "Littoral rock",
    18: "Littoral sediment",
    19: "Saltmarsh",
    20: "Urban",
    21: "Suburban"
}

# Batch coordinate transformation
transformer_ni = Transformer.from_crs("EPSG:4326", "EPSG:29903", always_xy=True)
transformer_gb = Transformer.from_crs("EPSG:4326", "EPSG:27700", always_xy=True)

coords = list(zip(df['decimalLongitude'], df['decimalLatitude']))
df['easting_ni'], df['northing_ni'] = zip(*transformer_ni.itransform(coords))
df['easting_gb'], df['northing_gb'] = zip(*transformer_gb.itransform(coords))

# Raster processing optimization
gb_raster = 'gblcm2023_10m.tif'
n_ireland_raster = 'nilcm2023_10m.tif'


def get_land_cover_class(row):
    try:
        # Try GB raster first
        with rasterio.open(gb_raster) as src:
            row_idx, col_idx = src.index(row['easting_gb'], row['northing_gb'])
            # Read a small window around the point for better performance
            window = Window(col_idx, row_idx, 1, 1)
            land_cover_class = src.read(1, window=window)[0, 0]

            if land_cover_class == 0:  # Check NI raster if GB is 0
                with rasterio.open(n_ireland_raster) as src_ni:
                    row_idx, col_idx = src_ni.index(row['easting_ni'], row['northing_ni'])
                    window = Window(col_idx, row_idx, 1, 1)
                    land_cover_class = src_ni.read(1, window=window)[0, 0]

        return land_cover_map.get(land_cover_class, "Unknown")
    except Exception as e:
        print(f"Error processing row: {e}")
        return "Unknown"


df['Land_cover'] = df.apply(get_land_cover_class, axis=1)


Error processing row: index 0 is out of bounds for axis 0 with size 0
Error processing row: index 0 is out of bounds for axis 0 with size 0


,species,decimalLatitude,decimalLongitude,easting_ni,northing_ni,easting_gb,northing_gb,Land_cover
0,Erinaceus europaeus,51.404820,-3.530218,511021.330416,26315.705291,293658.538615,168421.657658,Suburban
1,Erinaceus europaeus,52.170547,1.340593,838414.648219,143288.339264,628532.635367,257737.644843,Improved grassland
2,Erinaceus europaeus,50.851716,-0.908066,699230.107370,-20693.893184,476962.212740,106367.644364,Suburban
3,Erinaceus europaeus,52.326169,1.389178,839471.124825,160977.261328,631043.898189,275194.263768,Suburban
4,Erinaceus europaeus,53.428633,-2.267750,580902.881872,257357.931662,382306.562378,392460.548262,Suburban
5,Sciurus vulgaris,54.536084,-2.957853,526302.070431,377008.907159,338119.601033,516068.110970,Deciduous woodland
6,Sciurus vulgaris,50.744077,-1.405282,665337.134331,-35906.763266,442057.675930,93998.553064,Unknown
7,Sciurus vulgaris,56.174384,-3.956572,451120.531054,555066.667426,278626.822958,699694.080280,Improved grassland
8,Sciurus vulgaris,57.338583,-3.751360,455807.552167,685335.599910,294679.427270,828919.298024,Heather
9,Sciurus vulgaris,50.610281,-1.496538,660215.254772,-51343.822626,435720.070829,79072.680100,Unknown


In [31]:
from geopy.distance import geodesic

def get_aoi(center_coords, box_size_km):
    """
    Create a 1km x 1km bounding box centered around a given point using geodesic distances.

    Args:
        center_coords (list): A list containing the center point coordinates [lon, lat].
        box_size_km (float): Size of the bounding box in kilometers (default is 1km).

    Returns:
        list: A list of coordinates representing the bounding box polygon.
              Format: [[lon1, lat1], [lon2, lat2], [lon3, lat3], [lon4, lat4], [lon1, lat1]]
    """
    # Extract longitude and latitude from the center coordinates
    lon, lat = center_coords

    # Calculate the distance in kilometers for half the box size
    half_size_km = box_size_km / 2

    # Calculate the four corners of the bounding box using geodesic distances
    north = geodesic(kilometers=half_size_km).destination((lat, lon), bearing=0)  # North
    south = geodesic(kilometers=half_size_km).destination((lat, lon), bearing=180)  # South
    east = geodesic(kilometers=half_size_km).destination((lat, lon), bearing=90)  # East
    west = geodesic(kilometers=half_size_km).destination((lat, lon), bearing=270)  # West

    # Extract coordinates of the corners
    top_left = [west.longitude, north.latitude]
    top_right = [east.longitude, north.latitude]
    bottom_right = [east.longitude, south.latitude]
    bottom_left = [west.longitude, south.latitude]

    # Define the bounding box polygon (clockwise or counter-clockwise)
    bounding_box = [
        bottom_left,  # Bottom-left corner
        bottom_right,  # Bottom-right corner
        top_right,  # Top-right corner
        top_left,  # Top-left corner
        bottom_left  # Close the polygon
    ]

    aoi = ee.Geometry.Polygon(bounding_box)

    return aoi

def get_indices(aoi, start_date, end_date):
  dataset = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")  # Sentinel-2 Surface Reflectance
  filtered = dataset.filterBounds(aoi).filterDate(start_date, end_date).sort('CLOUDY_PIXEL_PERCENTAGE').first()

  # Clip the image to the AOI
  clipped_image = filtered.clip(aoi)

  # Extract bands (e.g., B2, B3, B4, B8 for Sentinel-2)
  blue_band = clipped_image.select('B2')  # Blue band
  green_band = clipped_image.select('B3')  # Green band
  red_band = clipped_image.select('B4')   # Red band
  nir_band = clipped_image.select('B8')   # Near-Infrared band
  swir_band = clipped_image.select('B11') # Short-Wave Infrared band

  # Constants for SAVI and EVI
  # Constants for SAVI and EVI
  L = ee.Number(0.5)  # Constant for SAVI

  # Compute NDVI
  ndvi = nir_band.subtract(red_band).divide(nir_band.add(red_band)).rename('NDVI')

  # Compute NDWI
  ndwi = green_band.subtract(nir_band).divide(green_band.add(nir_band)).rename('NDWI')

  # Compute NDBI
  ndbi = swir_band.subtract(nir_band).divide(swir_band.add(nir_band)).rename('NDBI')

  # Compute SAVI
  savi = nir_band.subtract(red_band).divide(nir_band.add(red_band).add(L)).multiply(ee.Number(1).add(L)).rename('SAVI')

  # Compute MNDWI
  mndwi = green_band.subtract(swir_band).divide(green_band.add(swir_band)).rename('MNDWI')

  ndsi = green_band.subtract(swir_band).divide(green_band.add(swir_band)).rename('NDSI')

  bsi = (red_band.add(blue_band).subtract(nir_band.add(swir_band))).divide(red_band.add(blue_band).add(nir_band.add(swir_band))).rename('BSI')

  # Compute NDBI (Already included in previous code)
  ndbi = swir_band.subtract(nir_band).divide(swir_band.add(nir_band)).rename('NDBI')

  # Compute UI (Urban Index)
  ui = nir_band.subtract(swir_band).divide(nir_band.add(swir_band)).rename('UI')

  # Compute mean value for each index over the polygon
  ndvi_mean = ndvi.reduceRegion(
      reducer=ee.Reducer.mean(),
      geometry=aoi,
      scale=10
  ).get('NDVI').getInfo()

  ndwi_mean = ndwi.reduceRegion(
      reducer=ee.Reducer.mean(),
      geometry=aoi,
      scale=10
  ).get('NDWI').getInfo()

  ndbi_mean = ndbi.reduceRegion(
      reducer=ee.Reducer.mean(),
      geometry=aoi,
      scale=10
  ).get('NDBI').getInfo()

  savi_mean = savi.reduceRegion(
      reducer=ee.Reducer.mean(),
      geometry=aoi,
      scale=10
  ).get('SAVI').getInfo()

  mndwi_mean = mndwi.reduceRegion(
      reducer=ee.Reducer.mean(),
      geometry=aoi,
      scale=10
  ).get('MNDWI').getInfo()

  ndsi_mean = ndsi.reduceRegion(
      reducer=ee.Reducer.mean(),
      geometry=aoi,
      scale=10
  ).get('NDSI').getInfo()

  bsi_mean = bsi.reduceRegion(
      reducer=ee.Reducer.mean(),
      geometry=aoi,
      scale=10
  ).get('BSI').getInfo()

  ndbi_mean = ndbi.reduceRegion(
      reducer=ee.Reducer.mean(),
      geometry=aoi,
      scale=10
  ).get('NDBI').getInfo()

  ui_mean = ui.reduceRegion(
      reducer=ee.Reducer.mean(),
      geometry=aoi,
      scale=10
  ).get('UI').getInfo()

  return round(ndvi_mean,2), round(ndwi_mean,2), round(ndbi_mean,2), round(savi_mean,2), round(mndwi_mean,2), round(ndsi_mean,2), round(bsi_mean,2), round(ndbi_mean,2), round(ui_mean,2)

def get_LST(aoi, start_date, end_date):

  # Import the MODIS LST dataset
  dataset = ee.ImageCollection("MODIS/061/MOD11A1")

  # Filter the dataset by date and AOI, then select the first image
  filtered = dataset.filterBounds(aoi).filterDate(start_date, end_date)
  # Calculate mean LST over the entire image collection
  lst_collection = filtered.select('LST_Day_1km').map(lambda image: image.multiply(0.02))

  # Reduce the collection by calculating the mean
  mean_lst_over_time = lst_collection.mean()

  # Clip and calculate mean LST over the AOI
  mean_lst = mean_lst_over_time.reduceRegion(
      reducer=ee.Reducer.mean(),
      geometry=aoi,
      scale=1000  # 1 km resolution
  )

  # Print the mean LST value
  mean_lst_value = mean_lst.getInfo()["LST_Day_1km"]
  return round(mean_lst_value,2)

In [32]:
from datetime import datetime
from datetime import timedelta
today = datetime.today().strftime('%Y-%m-%d')
start_date = (datetime.today() - timedelta(days=15)).strftime('%Y-%m-%d')

df["NDVI"] = 0.0
df["NDWI"] = 0.0
df["NDBI"] = 0.0
df["SAVI"] = 0.0
df["MNDWI"] = 0.0
df["NDSI"] = 0.0
df["BSI"] = 0.0
df["NDBI"] = 0.0
df["UI"] = 0.0
df["LST"] = 0.0

for index, row in tqdm(df.iterrows()):
  try:
    coords = [row['decimalLongitude'], row['decimalLatitude']]
    aoi = get_aoi(center_coords = coords, box_size_km=1)
    indices = get_indices(aoi, start_date, today)
    lst = get_LST(aoi, start_date, today)
    df.at[index, 'NDVI'] = indices[0]
    df.at[index, 'NDWI'] = indices[1]
    df.at[index, 'NDBI'] = indices[2]
    df.at[index, 'SAVI'] = indices[3]
    df.at[index, 'MNDWI'] = indices[4]
    df.at[index, 'NDSI'] = indices[5]
    df.at[index, 'BSI'] = indices[6]
    df.at[index, 'NDBI'] = indices[7]
    df.at[index, 'UI'] = indices[8]
    df.at[index, 'LST'] = lst
  except Exception as e:
    print(e)
    pass

5it [00:32,  6.45s/it]


In [33]:
df.head()

,species,decimalLatitude,decimalLongitude,NDVI,NDWI,NDBI,SAVI,MNDWI,NDSI,BSI,UI,LST
0,Muscardinus avellanarius,52.517983,0.872556,0.56,-0.58,-0.14,0.84,-0.50,-0.50,-0.61,0.14,296.83
1,Muscardinus avellanarius,52.685929,1.327543,0.48,-0.50,-0.08,0.72,-0.45,-0.45,-0.54,0.08,297.17
2,Muscardinus avellanarius,52.587545,1.615422,0.75,-0.68,-0.27,1.13,-0.50,-0.50,-0.72,0.27,295.60
3,Muscardinus avellanarius,52.607768,0.878431,0.73,-0.68,-0.31,1.09,-0.47,-0.47,-0.70,0.31,295.65
4,Muscardinus avellanarius,52.591956,1.468096,0.76,-0.67,-0.34,1.14,-0.45,-0.45,-0.72,0.34,294.24


In [29]:
df.head()

,species,decimalLatitude,decimalLongitude,NDVI,NDWI,NDBI,SAVI,MNDWI,NDSI,BSI,UI,LST
0,Muscardinus avellanarius,52.517983,0.872556,0.56,-0.58,-0.14,0.84,-0.50,-0.50,-0.61,0.14,296.83
1,Muscardinus avellanarius,52.685929,1.327543,0.48,-0.50,-0.08,0.72,-0.45,-0.45,-0.54,0.08,297.17
2,Muscardinus avellanarius,52.587545,1.615422,0.75,-0.68,-0.27,1.13,-0.50,-0.50,-0.72,0.27,295.60
3,Muscardinus avellanarius,52.607768,0.878431,0.73,-0.68,-0.31,1.09,-0.47,-0.47,-0.70,0.31,295.65
4,Muscardinus avellanarius,52.591956,1.468096,0.76,-0.67,-0.34,1.14,-0.45,-0.45,-0.72,0.34,294.24


In [44]:
df.to_csv('data.csv', index=False)

In [51]:
import pandas as pd
df=pd.read_csv('data.csv')
df=df.dropna()
# Landcover is categorical data, so dummies
df = pd.get_dummies(df, columns=['Land_cover'])

In [52]:
# class is species

#predict class using MLP
from sklearn.calibration import CalibratedClassifierCV
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler

X = df.drop(columns=['species', 'decimalLatitude', 'decimalLongitude'])
y = df['species']

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = MLPClassifier(random_state=1, max_iter=500).fit(X_train, y_train)
y_pred = clf.predict(X_test)

#print prob of test[0]
print(clf.predict_proba([X_test[0]]))

[[0.22210512 0.7028286  0.07506628]]


/Users/arzaan/PycharmProjects/ComputationalIntelligence/venv/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


In [53]:
# classification report
print(classification_report(y_test, y_pred))



                     precision    recall  f1-score   support

Erinaceus europaeus       0.82      0.67      0.74        63
   Sciurus vulgaris       0.80      0.78      0.79        65
      Vulpes vulpes       0.66      0.83      0.74        52

           accuracy                           0.76       180
          macro avg       0.76      0.76      0.75       180
       weighted avg       0.77      0.76      0.76       180


In [54]:
# class is species
from sklearn.calibration import CalibratedClassifierCV


#predict class using MLP
from sklearn.calibration import CalibratedClassifierCV
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler

X = df.drop(columns=['species', 'decimalLatitude', 'decimalLongitude'])
y = df['species']

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = MLPClassifier(random_state=1, max_iter=500).fit(X_train, y_train)


# Calibrate classifier (sigmoid calibration works better for small datasets)
calibrated_clf = CalibratedClassifierCV(clf, method='sigmoid', cv='prefit')
calibrated_clf.fit(X_train, y_train)

y_pred = calibrated_clf.predict(X_test)
print(classification_report(y_test, y_pred))

                     precision    recall  f1-score   support

Erinaceus europaeus       0.82      0.67      0.74        63
   Sciurus vulgaris       0.80      0.78      0.79        65
      Vulpes vulpes       0.66      0.83      0.74        52

           accuracy                           0.76       180
          macro avg       0.76      0.76      0.75       180
       weighted avg       0.77      0.76      0.76       180


/Users/arzaan/PycharmProjects/ComputationalIntelligence/venv/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/arzaan/PycharmProjects/ComputationalIntelligence/venv/lib/python3.12/site-packages/sklearn/calibration.py:333: UserWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(


In [55]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.multiclass import OneVsRestClassifier

# Preprocessing
X = df.drop(columns=['species', 'decimalLatitude', 'decimalLongitude'])
y = df['species']

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# One-vs-Rest Classifier (Independent Class Probabilities)
clf = OneVsRestClassifier(MLPClassifier(random_state=1, max_iter=500))
clf.fit(X_train, y_train)

# Get independent probabilities for each class
y_probs = clf.predict_proba(X_test)

# Print each class probability for the first test sample
for i, class_name in enumerate(clf.classes_):
    print(f"{class_name}: {y_probs[0][i]:.4f}")


/Users/arzaan/PycharmProjects/ComputationalIntelligence/venv/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/arzaan/PycharmProjects/ComputationalIntelligence/venv/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Erinaceus europaeus: 0.2525
Sciurus vulgaris: 0.6740
Vulpes vulpes: 0.0735


/Users/arzaan/PycharmProjects/ComputationalIntelligence/venv/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


In [58]:
for i, class_name in enumerate(clf.classes_):
    print(f"{class_name}: {y_probs[3][i]:.4f}")

Erinaceus europaeus: 0.2182
Sciurus vulgaris: 0.0377
Vulpes vulpes: 0.7441


In [3]:
import rasterio

# Open the land cover map
lcm_path = "nilcm2023_10m.tif"
src = rasterio.open(lcm_path)

# Check CRS
print("CRS:", src.crs)

from pyproj import Transformer

# Example: WGS84 → Irish Grid
transformer = Transformer.from_crs("EPSG:4326", src.crs, always_xy=True)

# Sample lat/lon coordinate
lon, lat = -8.55, 53.34
x, y = transformer.transform(lon, lat)
print("Reprojected coordinate:", x, y)


CRS: EPSG:29903
Reprojected coordinate: 163413.22870545922 232309.61159488405


In [5]:
import rasterio

# Open raster
with rasterio.open("nilcm2023_10m.tif") as src:
    # Coordinate must match CRS (e.g. EPSG:29903 for Irish Grid)
    # x, y = 200000, 350000  # Example coordinate in Irish Grid
    value = list(src.sample([(x, y)]))[0][0]
    print("Land Cover Code:", value)


Land Cover Code: 0


In [14]:
import rasterio

tiff_path = 'nilcm2023_10m.tif'

with rasterio.open(tiff_path) as src:
    print("Number of bands:", src.count)
    print("CRS:", src.crs)
    print("Band descriptions:")
    for i in range(1, src.count + 1):
        print(f" Band {i} description: {src.descriptions[i-1]}")

    # You can also read stats for the first band
    band1 = src.read(1)
    print("Band 1 value range:", band1.min(), "-", band1.max())
    if src.count >= 2:
        band2 = src.read(2)
        print(f"Band 2 value range: min={band2.min()}, max={band2.max()}")
    else:
        print("The raster does not have a band 2.")



Number of bands: 2
CRS: EPSG:29903
Band descriptions:
 Band 1 description: None
 Band 2 description: None
Band 1 value range: 0 - 21
Band 2 value range: min=0, max=100


In [11]:
import rasterio

# Coordinates for Belfast
lat, lon = 54.5973, -5.9301

# Path to your TIFF
tiff_path = 'nilcm2023_10m.tif'

with rasterio.open(tiff_path) as src:
    # Since CRS is EPSG:4326, no coordinate transform needed
    # Get pixel row/col indices from lon/lat (x/y)
    row, col = src.index(lon, lat)
    print(f"Pixel indices: row={row}, col={col}")

    # Read the first band (land cover class)
    land_cover_class = src.read(1)[row, col]
    print(f"Land cover class at ({lat}, {lon}): {land_cover_class}")


Pixel indices: row=49994, col=-18001


IndexError: index 49994 is out of bounds for axis 0 with size 20000

In [12]:
import rasterio
from rasterio.coords import BoundingBox

lat, lon = 54.5973, -5.9301
tiff_path = 'nilcm2023_10m.tif'

with rasterio.open(tiff_path) as src:
    bounds: BoundingBox = src.bounds
    print("Raster bounds:", bounds)

    # Check if the point is inside the raster
    if bounds.left <= lon <= bounds.right and bounds.bottom <= lat <= bounds.top:
        row, col = src.index(lon, lat)
        print(f"Pixel indices: row={row}, col={col}")
        land_cover_class = src.read(1)[row, col]
        print(f"Land cover class at ({lat}, {lon}): {land_cover_class}")
    else:
        print(f"Point ({lat}, {lon}) is outside the raster bounds.")


Raster bounds: BoundingBox(left=180000.0, bottom=300000.0, right=400000.0, top=500000.0)
Point (54.5973, -5.9301) is outside the raster bounds.


In [13]:
if src.count >= 2:
    band2 = src.read(2)
    print(f"Band 2 value range: min={band2.min()}, max={band2.max()}")
else:
    print("The raster does not have a band 2.")


RasterioIOError: Dataset is closed: nilcm2023_10m.tif

In [18]:
from pyproj import Transformer
import rasterio

# Belfast lat/lon
lat, lon = 54.1751, -6.3402

# Set up coordinate transformer: WGS84 → British National Grid (EPSG:27700)
transformer = Transformer.from_crs("EPSG:4326", "EPSG:29903", always_xy=True)
easting, northing = transformer.transform(lon, lat)

print(f"Converted to BNG: easting={easting}, northing={northing}")

tiff_path = 'nilcm2023_10m.tif'

with rasterio.open(tiff_path) as src:
    # Now get row/col using BNG coordinates
    row, col = src.index(easting, northing)
    print(f"Pixel indices: row={row}, col={col}")
    
    land_cover_class = src.read(1)[row, col]
    print(f"Land cover class at Belfast: {type(land_cover_class)}")

    # Optional: Band 2 range
    if src.count >= 2:
        band2 = src.read(2)
        print(f"Band 2 range: {band2.min()} to {band2.max()}")


Converted to BNG: easting=308433.0804550406, northing=326392.6827254206
Pixel indices: row=17360, col=12843
Land cover class at Belfast: <class 'numpy.uint8'>
Band 2 range: 0 to 100


In [13]:
import osmnx as ox
from shapely.geometry import Point
from shapely.ops import nearest_points
from geopy.distance import geodesic

lat = 52.97043
lon = 0.753532

try:
    G = ox.graph_from_point((lat, lon), dist=5000, network_type='drive_service')
    edges = ox.graph_to_gdfs(G, nodes=False, edges=True)
    
    # original point
    pt = Point(lon, lat)
    
    # find nearest road in lon/lat
    nearest_geom = nearest_points(pt, edges.unary_union)[1]
    nearest_lon, nearest_lat = nearest_geom.x, nearest_geom.y
    
    # compute real‐world distance
    distance_m = geodesic((lat, lon), (nearest_lat, nearest_lon)).meters
    print(f"Distance to nearest road: {distance_m:.2f} m")
except ValueError as e:
    print(f"No roads: {e}")



Distance to nearest road: 724.77 m


/var/folders/0n/sggn66691kd6g5mtyzpq2hv80000gn/T/ipykernel_18706/2031346650.py:17: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  nearest_geom = nearest_points(pt, edges.unary_union)[1]


In [20]:
import osmnx as ox
import networkx as nx
from shapely.geometry import Point
import geopandas as gpd

# Your list of (lat, lon) tuples
coords = [
    (52.97043, 0.753532),
    (52.97100, 0.750000),
    # add more...
]

# Extract lats and lons separately
lats = [lat for lat, lon in coords]
lons = [lon for lat, lon in coords]

# Define bounding box with buffer
buffer = 0.02
north, south = max(lats) + buffer, min(lats) - buffer
east, west = max(lons) + buffer, min(lons) - buffer

bbox = (west, south, east, north)
# If your osmnx version is older, adjust accordingly (see your docs)
G = ox.graph_from_bbox(bbox, network_type='drive_service')

# Project graph to UTM (meters)
G_proj = ox.project_graph(G)

# Prepare projected nodes GeoDataFrame for distance calculations
nodes_proj, edges_proj = ox.graph_to_gdfs(G_proj)

results = []
for lat, lon in coords:
    # Project the point to the graph's CRS (meters)
    point_geom = Point(lon, lat)
    point_gdf = gpd.GeoSeries([point_geom], crs='EPSG:4326')
    point_proj = point_gdf.to_crs(nodes_proj.crs).iloc[0]

    # Use osmnx's nearest_edges (input projected xy coords)
    nearest_edge = ox.distance.nearest_edges(G_proj, [point_proj.x], [point_proj.y])[0]  # returns (u,v,key)
    
    # Get geometry of the edge
    edge_geom = edges_proj.loc[nearest_edge]['geometry']

    # Calculate planar distance from point to edge geometry in meters
    distance = point_proj.distance(edge_geom)
    results.append(distance)

# Print results
for coord, dist in zip(coords, results):
    print(f"Point {coord} is {dist:.2f} meters from nearest road")


Point (52.97043, 0.753532) is 724.68 meters from nearest road
Point (52.971, 0.75) is 760.62 meters from nearest road
